In [21]:
#import all essential libraries
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.metrics import mean_squared_error,r2_score
## Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers import Dropout

In [22]:
df=pd.read_excel('Book01.xlsx')
df.head()

,Date,MB033,MB034,MB035,MB036
0,2021-01-01,site off,4,71,40
1,2021-01-02,88,66,52,12
2,2021-01-03,52,40,54,72
3,2021-01-04,49,34,32,28
4,2021-01-05,82,32,85,49


In [23]:
df.tail()

,Date,MB033,MB034,MB035,MB036
495,2022-05-11,70,42,6,2
496,2022-05-12,92,3,99,64
497,2022-05-13,84,82,24,30
498,2022-05-14,98,50,19,60
499,2022-05-15,3,0,66,48


In [24]:
#check for missing values in train data
df.isnull().sum()
 #No missing valuues



Date     0
MB033    0
MB034    0
MB035    0
MB036    0
dtype: int64

Here for better understanding of the data, We can eloborate as month and weekday wise.

In [25]:
df['MB035']

0      71
1      52
2      54
3      32
4      85
       ..
495     6
496    99
497    24
498    19
499    66
Name: MB035, Length: 500, dtype: object

In [26]:
df['MB033'] = df['MB033'].replace(['site off'],0)
df['MB034'] = df['MB034'].replace(['site off'],0)
df['MB035'] = df['MB035'].replace(['site off'],0)
df['MB036'] = df['MB036'].replace(['site off'],0)









In [27]:
arr1 = df['MB035'].values
arr1

array([ 71,  52,  54,  32,  85,  17,  71,  36,  16,  68,  12,   4,  68,
        76,  38,  25,  38,  96,  97,  78,  73,  58,  11,  99,  38,  79,
        17,  41,  13,  99,   0,  47,  99,  46,  70,  64,  64,  61,  21,
         1,  78,   1, 100,  49,  42,  64,  73,   4,  95,  80,   7,  76,
        72,  91,  63,  80,  80,  64,   0,  34,  16,   8,  28,  19,  55,
        84,  82,  79,  44,  73,  99,  79,  65,  27,  62,  40,  48,  27,
        83,  24,  13,  45,  53,   0,  69,  22,  60,  85,  65,   0,  77,
        53,   2,  30,  93,  22,   4,  16,  70,  70,  80,  40,  76,  38,
        89,  36,  92,  13,  29,  72,  87,  26,  75,   4,  89,  83,  53,
        92,  25,   0,  29,  93,  96,  23,  26,  62,  58,  54,  19,  47,
        64,  98,   3,  90,  53,  53,  10,  51,  14,   1,  50,  71,  60,
        89,  82,   9,  43,  12,  39,  73,   0,  48,  18,  47,  98,  73,
        43,  87,  11,  11,  98,  82,  46,  92,  17,  48,  64,  95,  61,
        82,  36,  60,  31,  63,  59,  73,  32,  72,  24,  60,   

In [28]:
import numpy as np
def splitSequence(seq, n_steps):
    
    #Declare X and y as empty list
    X = []
    y = []
    
    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps
        
        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break
            
        #Create input and output sequence
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]
        
        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        pass
    #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)
    
    return X,y 
    
    pass

n_steps = 15
X, y = splitSequence(arr1, n_steps = 15)

In [29]:
X.shape,y.shape

((485, 15), (485,))

In [30]:
for i in range(len(X)):
    print(X[i], y[i])

[71 52 54 32 85 17 71 36 16 68 12  4 68 76 38] 25
[52 54 32 85 17 71 36 16 68 12  4 68 76 38 25] 38
[54 32 85 17 71 36 16 68 12  4 68 76 38 25 38] 96
[32 85 17 71 36 16 68 12  4 68 76 38 25 38 96] 97
[85 17 71 36 16 68 12  4 68 76 38 25 38 96 97] 78
[17 71 36 16 68 12  4 68 76 38 25 38 96 97 78] 73
[71 36 16 68 12  4 68 76 38 25 38 96 97 78 73] 58
[36 16 68 12  4 68 76 38 25 38 96 97 78 73 58] 11
[16 68 12  4 68 76 38 25 38 96 97 78 73 58 11] 99
[68 12  4 68 76 38 25 38 96 97 78 73 58 11 99] 38
[12  4 68 76 38 25 38 96 97 78 73 58 11 99 38] 79
[ 4 68 76 38 25 38 96 97 78 73 58 11 99 38 79] 17
[68 76 38 25 38 96 97 78 73 58 11 99 38 79 17] 41
[76 38 25 38 96 97 78 73 58 11 99 38 79 17 41] 13
[38 25 38 96 97 78 73 58 11 99 38 79 17 41 13] 99
[25 38 96 97 78 73 58 11 99 38 79 17 41 13 99] 0
[38 96 97 78 73 58 11 99 38 79 17 41 13 99  0] 47
[96 97 78 73 58 11 99 38 79 17 41 13 99  0 47] 99
[97 78 73 58 11 99 38 79 17 41 13 99  0 47 99] 46
[78 73 58 11 99 38 79 17 41 13 99  0 47 99 46] 70
[

In [31]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X[:2])

[[[71]
  [52]
  [54]
  [32]
  [85]
  [17]
  [71]
  [36]
  [16]
  [68]
  [12]
  [ 4]
  [68]
  [76]
  [38]]

 [[52]
  [54]
  [32]
  [85]
  [17]
  [71]
  [36]
  [16]
  [68]
  [12]
  [ 4]
  [68]
  [76]
  [38]
  [25]]]


In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [33]:
model3 = tf.keras.Sequential()
model3.add(layers.LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model3.add(layers.Dense(1))

In [34]:
model3.layers

In [35]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [36]:
model3.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [37]:
model3.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
16/16 [==============================] - 1s 11ms/step - loss: 1651.1329 - accuracy: 0.0124
Epoch 2/200
16/16 [==============================] - 0s 10ms/step - loss: 1244.8685 - accuracy: 0.0124
Epoch 3/200
16/16 [==============================] - 0s 10ms/step - loss: 1022.9379 - accuracy: 0.0124
Epoch 4/200
16/16 [==============================] - 0s 10ms/step - loss: 1007.8016 - accuracy: 0.0124
Epoch 5/200
16/16 [==============================] - 0s 10ms/step - loss: 1004.9525 - accuracy: 0.0124
Epoch 6/200
16/16 [==============================] - 0s 9ms/step - loss: 989.6764 - accuracy: 0.0124
Epoch 7/200
16/16 [==============================] - 0s 10ms/step - loss: 988.6370 - accuracy: 0.0124
Epoch 8/200
16/16 [==============================] - 0s 9ms/step - loss: 973.9019 - accuracy: 0.0124
Epoch 9/200
16/16 [==============================] - 0s 9ms/step - loss: 977.2203 - accuracy: 0.0124
Epoch 10/200
16/16 [==============================] - 0s 10ms/step - loss: 990.6

In [38]:
test_data = arr1[-15:]
test_data = test_data.reshape((1, n_steps, n_features))
test_data

array([[[ 88],
        [ 34],
        [ 19],
        [100],
        [ 73],
        [ 89],
        [ 59],
        [ 52],
        [ 20],
        [ 74],
        [  6],
        [ 99],
        [ 24],
        [ 19],
        [ 66]]])

In [39]:
test_data.mean()

54.8

In [40]:
predictNextNumber = model3.predict(test_data, verbose=1)
print(predictNextNumber)

1/1 [==============================] - 0s 220ms/step
[[46.275166]]
